In [1]:
import os
import sys
sys.path.append("/mnt/code")

In [2]:
import os
os.environ['DOMINO_STARTING_USERNAME']

'integration-test'

In [4]:
#Just some imports you will need
import mlflow
import jwt
import json
import warnings
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

### How does MLFlow know the URI of Tracking Server and the Authentication Token?
MLFlow relies on two environment variables
- `MLFLOW_TRACKING_URI` - Which is used to find the location of the tracking uri
- `MLFLOW_TRACKING_TOKEN` - The value of which is passed as an Authentication Bearer token to every Rest call from the invoking Python/Java/R API. With the Domino Experiment Manager, this token is generated on your behalf by the underlying Domino Infrastructure when a call is made from the workspace. Note how the `MLFLOW_TRACKING_URI` is a `localhost` call and is running as side-car in your workload pod. It is responsible for invoking the MLflow proxy with this token. You will not have access to this token from inside the workspace

In [5]:
print('MLFLOW_TRACKING_URI: ' + os.environ['MLFLOW_TRACKING_URI'])

MLFLOW_TRACKING_URI: http://127.0.0.1:8765


### Create MLFlow Client
We are ready to start using MLFLOW. Let us create an MLFlow Client. Note that we do not provide any parameters. It knows the Tracking URI and our Authentication Token from the environment variables created by the workspace environment.

> Your credentials are passed transparently. No configuration is needed from the user end

In [6]:
client = mlflow.tracking.MlflowClient()

### Create a new experiment or use an existing one

Provide a value for `myprefix` variable below which is unused to create a brand new 
experiment. Or simply leave it as it is. The experiment name will be the combination of 
- myprefix
- today's date
- user-name
- project-name

If the experiment with that name exists, you will be using it to create runs or a new experiment will be created and you will create runs into it

In [7]:
from datetime import datetime
now = datetime.now()
date_time_str = now.strftime("%m-%d-%Y")
prefix = 'DEMO-SCIKIT-LEARN-EXPERIMENT'
starting_user_name = os.environ['DOMINO_STARTING_USERNAME']
project_id = os.environ['DOMINO_PROJECT_ID']

experiment_name = f'{prefix}-{project_id}'
model_name = f'model-{prefix}'
print(experiment_name)

DEMO-SCIKIT-LEARN-EXPERIMENT-66b3618bffe62d0ab852cd05


In [ ]:
#MLFLOW Specific
experiment = client.get_experiment_by_name(name=experiment_name)
if(experiment is None):
    print('Creating experiment ')
    client.create_experiment(name=experiment_name)
    experiment = client.get_experiment_by_name(name=experiment_name)
print(experiment_name)
mlflow.set_experiment(experiment_name=experiment_name)

In [9]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [10]:
#MLFLOW Specific
mlflow.sklearn.autolog()

### Now multiple runs in that experiment for various values of alpha and l1_ratio


In [14]:
# Read the wine-quality csv file from the URL
csv_url = (
     "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
try:
     data = pd.read_csv(csv_url, sep=";")
except Exception as e:
     logger.exception(
          "Unable to download training & test CSV, check your internet connection. Error: %s", e
      )

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]


my_log = "This is a test log"
with open("/tmp/test.txt", 'w') as f:
    f.write(my_log)
with open("/tmp/test.log", 'w') as f:
    f.write(my_log)

    
    
#run_tags={'mlflow.user':os.environ['DOMINO_STARTING_USERNAME']}
#Change user name
alpha = 0.7
l1_ratio = 0.6
while(alpha<1):
    #Start a run in the experiment
    with mlflow.start_run():        
        print('Start Run')
        print('Alpha : ' + str(alpha))
        print('L1_Ratio : ' + str(l1_ratio))
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)
        mlflow.log_metric('fake_metric',0.46)
        mlflow.sklearn.log_model(lr,  model_name)
        alpha=alpha+0.1
        l1_ratio = l1_ratio + 0.05        
        

Start Run
Alpha : 0.7
L1_Ratio : 0.6
Elasticnet model (alpha=0.700000, l1_ratio=0.600000):
  RMSE: 0.7627246800819057
  MAE: 0.6287888854224957
  R2: 0.03747693976498101
Start Run
Alpha : 0.7999999999999999
L1_Ratio : 0.65
Elasticnet model (alpha=0.800000, l1_ratio=0.650000):
  RMSE: 0.7638598543917334
  MAE: 0.6312333902950296
  R2: 0.03460973336968853
Start Run
Alpha : 0.8999999999999999
L1_Ratio : 0.7000000000000001
Elasticnet model (alpha=0.900000, l1_ratio=0.700000):
  RMSE: 0.7641712496114665
  MAE: 0.6322852069908985
  R2: 0.03382247066057176
Start Run
Alpha : 0.9999999999999999
L1_Ratio : 0.7500000000000001
Elasticnet model (alpha=1.000000, l1_ratio=0.750000):
  RMSE: 0.7642701552596672
  MAE: 0.6330621329833208
  R2: 0.033572352381476045


Go to the MLFlow UI and view the following:
- Experiment
- Runs in the experiment
- Compare runs for performance
- View model and its versions (One per run)

For each run of this experiment let us review the list of artifacts

In [ ]:
 def print_artifact_info(artifact):
    print(f"   artifact path: {artifact.path}, is_dir: {artifact.is_dir}")
    #print("is_dir: {}".format(artifact.is_dir))
    #print("size: {}".format(artifact.file_size))

In [ ]:
import pandas as pd
import json
e = mlflow.get_experiment_by_name(experiment_name)
print(e.artifact_location)
rl = mlflow.search_runs(e.experiment_id,output_format="list")

for r in rl:
    run_id=r.info.run_id
    print(f"Artifacts for run id {run_id}")
    ls = client.list_artifacts(run_id)
    for l in ls:
        print_artifact_info(l)

## Get the latest model version for each stage (None, Staging, Production, Archive)

In [ ]:
client =  mlflow.tracking.MlflowClient()
registered_model = client.get_registered_model(model_name)

versions = registered_model.latest_versions
#print(registered_model)
#print(f"Number of Model Versions ={versions}")
model_version=None

if(len(versions)>1):
    #print(versions[1])
    model_version = versions[1]
else:
    model_version = versions[0]
    
print(f"Most Recent Model Name={model_version.name} and Most Recent Model Version={model_version.version}")


### Downloading Artifacts Locally

This is essential when you are deploying a specific model version

In [ ]:
#print(model_version)
run_id=model_version.run_id
artifact_path=model_version.source
#print(artifact_path)
home_dir = os.path.expanduser('~')
download_path=f'{home_dir}/{run_id}'
mlflow.artifacts.download_artifacts(run_id=run_id,dst_path=download_path)
print(f"The artifacts for {run_id} will be downloaded to the folder {download_path}")


In [ ]:
#! ls /home/ubuntu/c139836db5c44588a2b022c75e7fa991/model

### Fetch all model versions for a registered model (Not just the latest one as shown above)

In [ ]:

filter_string=f"name='{model_name}'"
results = mlflow.search_model_versions(filter_string=filter_string)
print(results)

In [ ]:
!jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace *.ipynb